In [1]:
!pip install gluonnlp==0.10.0 pandas tqdm   
!pip install mxnet
!pip install sentencepiece==0.1.91
!pip install transformers==4.8.2
!pip install torch==1.8.2

     |████████████████████████████████| 344 kB 5.1 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595737 sha256=6bd4e0a5a226796f8380ecb17bfb59ec6a0dc898cb6899cda5ff573d497b678c
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 46.9 MB 1.2 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 1.1 MB 5.2 MB/s 
     |████████████████████████████████| 2.5 MB 5.2 MB/s 
     |████████████████████████████████| 895 kB 73.1 MB/s 
     |████████████████████████████████| 3.3 MB 66.4 MB/s 
ERROR: Could not find a version that satisfies the requirement torch==1.8.2 (from versions: 0.4.1, 0.4.1.post2, 1.0.0, 1.0.1, 1.0.1.post2, 1.1.0, 1.2.0, 1.3.0, 1.3.1, 1.4.0, 1.5.0, 1.5.1, 1.

In [2]:
pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-ay_h9kso/kobert-tokenizer_acf908385ef142249293b9b18042b727
  Running command git clone -q https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-ay_h9kso/kobert-tokenizer_acf908385ef142249293b9b18042b727
  Created wheel for kobert-tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4682 sha256=ae1c0795f821ad4c2ad15aeba3824b8fbec6d0913d38e73e729ab60c28384c0d
  Stored in directory: /tmp/pip-ephem-wheel-cache-svraigsb/wheels/10/b4/d9/cb627bbfaefa266657b0b4e8127f7bf96d27376fa1a23897b4
Successfully built kobert-tokenizer


In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
#kobert
# from kobert.utils import get_tokenizer
# from kobert.pytorch_kobert import get_pytorch_kobert_model
# from kobert.pytorch_kobert import get_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from google.colab import drive
import sentencepiece
import transformers
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd
drive.mount('/content/drive')

Mounted at /content/drive


# 에러나서 다른 방법 사용

In [4]:
print(torch.__version__) #1.8.1
print(transformers.__version__) #4.8.2

1.10.0+cu111
4.8.2


In [5]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-nam8_7p1
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-nam8_7p1
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=12771 sha256=71a024ea2e62da463e562ddfd588edeeda289dfcf9416b8ff3940ac78ff0debe
  Stored in directory: /tmp/pip-ephem-wheel-cache-b90l7irh/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert


In [6]:
from kobert_tokenizer import KoBERTTokenizer
from kobert.pytorch_kobert import get_kobert_model
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

Downloading:   0%|          | 0.00/371k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/244 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/432 [00:00<?, ?B/s]

In [7]:
#띄어쓰기에 강건한 모델
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1', sp_model_kwargs={'nbest_size': -1, 'alpha': 0.6, 'enable_sampling': True})

In [8]:
import torch
from transformers import BertModel
# model = BertModel.from_pretrained('skt/kobert-base-v1')
# text = "한국어 모델을 공유합니다."
# inputs = tokenizer.batch_encode_plus([text])
# out = model(input_ids = torch.tensor(inputs['input_ids']),attention_mask = torch.tensor(inputs['attention_mask']))
# out.pooler_output.shape

In [9]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1', sp_model_kwargs={'nbest_size': -1, 'alpha': 0.6, 'enable_sampling': True})
bertmodel, vocab = get_kobert_model('skt/kobert-base-v1',tokenizer.vocab_file)

Downloading:   0%|          | 0.00/535 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/369M [00:00<?, ?B/s]

# 데이터 전처리

In [ ]:
# saturi = pd.read_csv('/content/drive/MyDrive/자연어처리 과제/all_사투리.csv')

In [ ]:
# import re
# data_list = []
# for q, label in zip(saturi['text'], saturi['label'])  :
#     data = []
#     q = str(q)
#     q = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', q)
#     q = re.sub('[A-Za-z]+_[A-Za-z]*','',q)
#     q = q.strip()
#     if len(q)>=20:
#         data.append(q)
#         data.append(str(label))

#         data_list.append(data)

In [ ]:
# import random

# data_list = random.sample(data_list, 150000)

In [ ]:
import pickle

with open("/content/drive/MyDrive/자연어처리 과제/model/15만개.txt", "rb") as fp:   # Unpickling
    data_list = pickle.load(fp)

In [ ]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, shuffle=True, random_state=34)

In [10]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
                 pad, pair):
   
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
         

    def __len__(self):
        return (len(self.labels))

In [ ]:
print(len(dataset_train))
print(len(dataset_test))

120000
30000


In [ ]:
print(data_list[0])
print(data_list[6000])
print(data_list[12000])
print(data_list[18000])
print(data_list[24000])
print(data_list[30000])
print(data_list[-1])
# np.unique(saturi['label'])

['할당량을 채워야 퇴근할 수 있는 거랜', '3']
['얘가 얘가 메칠   만에 아주 반짜가리가   났네', '0']
['가벼운 마음으로 볼 수 없으니까 이렇게 각을 잡고 봐야 된단 말이야 그래가주고 어저께는 이제', '2']
['하고 하는 사람들도 있어가지고 차라리 그냥 기졸업자 준비할 바에는', '2']
['어 차 종류 아이   에 대해서 이렇게 관심은 많으신가요', '1']
['그 뭐시기 그 여러 이 유튜브에서 뭐 먹방도 보고 그런 것도 보는디', '2']
['정말 쪼그만한   가족의 테두리인 거 같애', '2']


In [11]:
# Setting parameters
max_len = 64
batch_size = 128
warmup_ratio = 0.1
num_epochs = 2  
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

#토큰화 및 모델 돌리기

In [ ]:
#토큰화
tok=tokenizer.tokenize
data_train = BERTDataset(dataset_train, 0, 1, tok,vocab, max_len, True, False)
data_test = BERTDataset(dataset_test,0, 1, tok,vocab,  max_len, True, False)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [13]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=5,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [14]:
device = torch.device("cpu")

#BERT 모델 불러오기
# model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
model = torch.load('/content/drive/MyDrive/자연어처리 과제/model/사투리_지역분류_같은애로epoch8.h5')
 
#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

# t_total = len(train_dataloader) * num_epochs
# warmup_step = int(t_total * warmup_ratio)

# scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

# #정확도 측정을 위한 함수 정의
# def calc_accuracy(X,Y):
#     max_vals, max_indices = torch.max(X, 1)
#     train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
#     return train_acc
    
# train_dataloader

# 모델 학습

In [ ]:
train_history=[]
test_history=[]
loss_history=[]
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
         
        #print(label.shape,out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    #train_history.append(train_acc / (batch_id+1))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/938 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 0.5010454058647156 train acc 0.8125
epoch 1 batch id 201 loss 0.5029444694519043 train acc 0.804570895522388
epoch 1 batch id 401 loss 0.5964445471763611 train acc 0.796680174563591
epoch 1 batch id 601 loss 0.6790855526924133 train acc 0.7930922420965059
epoch 1 batch id 801 loss 0.6172838807106018 train acc 0.7903792134831461
epoch 1 train acc 0.7885044642857143


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 1 test acc 0.7773714539007092


  0%|          | 0/938 [00:00<?, ?it/s]

epoch 2 batch id 1 loss 0.5530635118484497 train acc 0.7578125
epoch 2 batch id 201 loss 0.43852052092552185 train acc 0.8121113184079602
epoch 2 batch id 401 loss 0.431906133890152 train acc 0.8198059538653366
epoch 2 batch id 601 loss 0.4534814953804016 train acc 0.8235882903494176
epoch 2 batch id 801 loss 0.45412522554397583 train acc 0.8240870786516854
epoch 2 train acc 0.8232775852878464


  0%|          | 0/235 [00:00<?, ?it/s]

epoch 2 test acc 0.7795988475177305


In [23]:
train_history=[]
test_history=[]
loss_history=[]
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
         
        #print(label.shape,out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    #train_history.append(train_acc / (batch_id+1))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/938 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 0.4507903456687927 train acc 0.8203125
epoch 1 batch id 201 loss 0.4531129002571106 train acc 0.8382695895522388
epoch 1 batch id 401 loss 0.47478270530700684 train acc 0.831553927680798
epoch 1 batch id 601 loss 0.5877231955528259 train acc 0.8259021422628952
epoch 1 batch id 801 loss 0.5215820670127869 train acc 0.8206636235955056
epoch 1 train acc 0.8176305970149254


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 1 test acc 0.7798537234042553


  0%|          | 0/938 [00:00<?, ?it/s]

epoch 2 batch id 1 loss 0.48140496015548706 train acc 0.8046875
epoch 2 batch id 201 loss 0.356849730014801 train acc 0.8442552860696517
epoch 2 batch id 401 loss 0.3306886851787567 train acc 0.8551083229426434
epoch 2 batch id 601 loss 0.3895307183265686 train acc 0.8568011647254575
epoch 2 batch id 801 loss 0.39344021677970886 train acc 0.8561953807740325
epoch 2 train acc 0.8543360207889126


  0%|          | 0/235 [00:00<?, ?it/s]

epoch 2 test acc 0.7766954787234043


In [24]:
torch.save(model, '/content/drive/MyDrive/자연어처리 과제/model/사투리_지역분류_같은애로epoch8.h5')

In [ ]:
import pickle
with open("/content/drive/MyDrive/자연어처리 과제/model/15만개.txt", "wb") as fp:   #Pickling
    pickle.dump(data_list, fp)

# 검증

In [15]:
#토큰화
# tokenizer = get_tokenizer()
# tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

eval_list = []
tok=tokenizer.tokenize

def predict(predict_sentence,label):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == int(label):
                eval_list.append(1)
            elif np.argmax(logits) != int(label):
                eval_list.append(0)

In [16]:
val_csv = pd.read_csv('/content/drive/MyDrive/자연어처리 과제/val_사투리.csv')
val_csv

,Unnamed: 0,text,label
0,0,강릉 사람들이 요새 코로나에 우터 대응하는지 니 아나?,0
1,1,잘 모르제 내가 야그할 테니 똑땍이 듣고 니도 따라 해이 돼.,0
2,2,언 우선 바깥에 댕겨오믄은,0
3,3,쭐쭐 흐르는 물에다가 비누를 거품 방구리가 방글방글 나도록,0
4,4,손을 싹싹 문대서 비베서 탁 씻거야 돼.,0
...,...,...,...
905286,905286,참 강아지가 이렇게 좀 쪼만한 거는 귀여운데,4
905287,905287,어머 -그- 그렇게 쪼만한데 어떻게 사람을 보고 그렇게 짖는지,4
905288,905288,이렇게 주먹만하게 아주 쪼만한 그런 미니 친구를 키우고 싶다는 생각을,4
905289,905289,꿈에 강아지 제가 하두 이렇게 쪼만한,4


In [20]:
np.unique(val_csv['label'])

array([0, 1, 2, 3, 4])

In [17]:
import re
val_list = []
for q, label in zip(val_csv['text'], val_csv['label'])  :
    data = []
    q = str(q)
    q = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', q)
    q = re.sub('[A-Za-z]+_[A-Za-z]*','',q)
    q = q.strip()
    if len(q)>=20:
        data.append(q)
        data.append(str(label))

        val_list.append(data)

In [18]:
import random

val_list = random.sample(val_list, 10000)

In [19]:
counter = 0
for i in val_list[:100]:
    predict(i[0], i[1])
    counter += 1
    print(counter)
print('전체 validation set에 대한 정확도는 {}입니다.'.format(sum(eval_list)))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
전체 validation set에 대한 정확도는 11입니다.
